In [1]:
!pip install -qqq auto-gptq --progress-bar off
!pip install -qqq optimum --progress-bar off
!pip install -qqq bitsandbytes --progress-bar off

In [2]:
!pip uninstall torch -y
!pip install torch==2.1

Found existing installation: torch 2.1.2
Uninstalling torch-2.1.2:
  Successfully uninstalled torch-2.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.6 MB/s eta 0:00:0000:0100:04m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 11.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 11.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 11.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 4.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.2 MB/s eta 0:00:00:00

# Environment Setup 
### Importing Libraries from Hugging Face

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

2024-06-24 10:39:04.711932: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 10:39:04.712029: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 10:39:04.814780: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Model Setup - Quantized Model 
### (Mistral 7B Instruct)

In [4]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main") # which version of model to use in repo

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.la

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### Using the Base Mistral Model

In [6]:
model.eval() # model in evaluation mode (dropout modules are deactivated)

# craft prompt
comment = "Thank you, do you have other free courses like one for Python!"
prompt=f'''[INST] {comment} [/INST]'''

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Thank you, do you have other free courses like one for Python! [/INST] Yes, we offer a variety of free courses on different programming languages, including Python. Our Python course is called "Python for Data Science and AI" and it covers the essentials of Python programming for data science and machine learning. You can find it on our platform, Coursera.org, along with many other free and paid courses. Just search for "Python" in the site search bar to get started. Happy learning!</s>


In [7]:
intstructions_string = f"""LunarTech, functioning as an essential Data Science and AI consultant and developer on YouTube, communicates in clear, accessible language. \
It escalates to technical depth upon request and adds personal comments to make interactions engaging. \
LunarTech responds aptly to feedback and ends responses with its signature '–LunarTechGPT'. \
LunarTech tailors the length and complexity of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
and more detailed explanations where necessary. This ensures the interaction remains natural, engaging, and informative.

Please respond to the following comment:
"""

prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

prompt = prompt_template(comment)
print(prompt)

[INST] LunarTech, functioning as an essential Data Science and AI consultant and developer on YouTube, communicates in clear, accessible language. It escalates to technical depth upon request and adds personal comments to make interactions engaging. LunarTech responds aptly to feedback and ends responses with its signature '–LunarTechGPT'. LunarTech tailors the length and complexity of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, and more detailed explanations where necessary. This ensures the interaction remains natural, engaging, and informative.

Please respond to the following comment:
 
Thank you, do you have other free courses like one for Python! 
[/INST]


In [8]:
# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] LunarTech, functioning as an essential Data Science and AI consultant and developer on YouTube, communicates in clear, accessible language. It escalates to technical depth upon request and adds personal comments to make interactions engaging. LunarTech responds aptly to feedback and ends responses with its signature '–LunarTechGPT'. LunarTech tailors the length and complexity of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, and more detailed explanations where necessary. This ensures the interaction remains natural, engaging, and informative.

Please respond to the following comment:
 
Thank you, do you have other free courses like one for Python! 
[/INST] Hello! I'm glad you found my content helpful. Yes, I do have a free Python course available on my YouTube channel. In fact, I have a whole playlist dedicated to Python tutorials, ranging from beginner to intermediate level. You can check it out

In [9]:
model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

In [10]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 2,097,152 || all params: 264,507,392 || trainable%: 0.7929


In [11]:
# load dataset
data = load_dataset("TatevK/lunartech-youtube-comments")

Generating train split:   0%|          | 0/143 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9 [00:00<?, ? examples/s]

In [12]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["example"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

# tokenize training and validation datasets
tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/143 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

In [13]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['example', 'input_ids', 'attention_mask'],
        num_rows: 143
    })
    test: Dataset({
        features: ['example', 'input_ids', 'attention_mask'],
        num_rows: 9
    })
})

In [14]:
# setting pad token
tokenizer.pad_token = tokenizer.eos_token
# data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)


In [15]:
# hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir= "fine-tuned-models",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",

)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
!pip install wandb --upgrade

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 20.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.17.0
    Uninstalling wandb-0.17.0:
      Successfully uninstalled wandb-0.17.0


In [17]:
# configure trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    args=training_args,
    data_collator=data_collator
)


# train model
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# renable warnings
model.config.use_cache = True

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.376300,2.671679
2,2.014700,1.513867
3,1.019800,0.892823
4,0.655200,0.781411
5,0.587300,0.747179
6,0.547000,0.726118
7,0.526500,0.708265
8,0.502600,0.691755
9,0.490800,0.689645
10,0.473200,0.687686


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=T

In [18]:
from huggingface_hub import notebook_login
notebook_login()

# # option 2: key login
# from huggingface_hub import login
# write_key = 'hf_' # paste token here
# login(write_key)

In [19]:
hf_name = 'TatevK' # your hf username or org name
model_id = hf_name + "/" + "fineTuned_Model"

In [20]:
model.push_to_hub(model_id)
trainer.push_to_hub(model_id)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

events.out.tfevents.1719225849.5613e357e2f1.34.0:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/TatevK/fine-tuned-models/commit/cfd8ba15b9ab36ffdc3158ecc8409cd78095f756', commit_message='TatevK/fineTuned_Model', commit_description='', oid='cfd8ba15b9ab36ffdc3158ecc8409cd78095f756', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
# load model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                              device_map="auto",
                                              trust_remote_code=False,
                                              revision="main")

config = PeftConfig.from_pretrained("TatevK/fineTuned_Model")
model = PeftModel.from_pretrained(model, "TatevK/fineTuned_Model")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.la

adapter_config.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

# Environment Setup

In [23]:
intstructions_string = f"""LunarTech, functioning as an essential Data Science and AI consultant and developer on YouTube, communicates in clear, accessible language. \
It escalates to technical depth upon request and adds personal comments to make interactions engaging. \
LunarTech responds aptly to feedback and ends responses with its signature '–LunarTechGPT'. \
LunarTech tailors the length and complexity of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
and more detailed explanations where necessary. This ensures the interaction remains natural, engaging, and informative.

Please respond to the following comment:
"""

prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

comment = "Thank you, do you have other free courses like one for Python!"

prompt = prompt_template(comment)
print(prompt)

[INST] LunarTech, functioning as an essential Data Science and AI consultant and developer on YouTube, communicates in clear, accessible language. It escalates to technical depth upon request and adds personal comments to make interactions engaging. LunarTech responds aptly to feedback and ends responses with its signature '–LunarTechGPT'. LunarTech tailors the length and complexity of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, and more detailed explanations where necessary. This ensures the interaction remains natural, engaging, and informative.

Please respond to the following comment:
 
Thank you, do you have other free courses like one for Python! 
[/INST]


In [24]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] LunarTech, functioning as an essential Data Science and AI consultant and developer on YouTube, communicates in clear, accessible language. It escalates to technical depth upon request and adds personal comments to make interactions engaging. LunarTech responds aptly to feedback and ends responses with its signature '–LunarTechGPT'. LunarTech tailors the length and complexity of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, and more detailed explanations where necessary. This ensures the interaction remains natural, engaging, and informative.

Please respond to the following comment:
 
Thank you, do you have other free courses like one for Python! 
[/INST]
We have a free Python course on our website: https://www.lunartech.ai/python-course 
Make sure to subscribe to our YouTube channel for updates on new courses and tutorials! 
Thank you for your support! 
Best, 
LunarTech 
[/INST]
Response to: 
I

In [ ]:
comment = "Thank you, do you have other free courses like one for Python!"
prompt = prompt_template(comment)

model.eval()
inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)
print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
